In [1]:
#importing required libraries
import pandas as pd
import numpy as np
import time
import re
import os
from bs4 import BeautifulSoup as bs
import json
from datetime import date
import requests

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys #imports keyboard keys
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
#selenium setup
options = webdriver.ChromeOptions()
options.add_argument("user-agent='Moxilla/5.0 (Linux; U; Android 2.2; en-us; T-Mobile G2 Build/FRF91) AppleWebKit/533.1(KHTML, like Gecko) Version/4.0 Mobile Safari/533.1'")
driver = webdriver.Chrome(r"C:\Users\Tejas\Downloads\chromedriver_win32\chromedriver.exe")#,chrome_options=options)
driver.set_window_size(1080,986)
# driver.maximize_window()

# driver.get("http://127.0.0.1:5000/forecast?region=Karnataka")
driver.get("https://www.mohfw.gov.in/")

In [4]:
table = pd.read_html(driver.page_source)[0]
table

S. No.  \
                                               S. No.   
0                                                   1   
1                                                   2   
2                                                   3   
3                                                   4   
4                                                   5   
5                                                   6   
6                                                   7   
7                                                   8   
8                                                   9   
9                                                  10   
10                                                 11   
11                                                 12   
12                                                 13   
13                                                 14   
14                                                 15   
15                                                 16   
16                                                 17   
17                                                 18   
18                                                 19   
19                                                 20   
20                                                 21   
21                                                 22   
22                                                 23   
23                                                 24   
24                                                 25   
25                                                 26   
26                                                 27   
27                                                 28   
28                                                 29   
29                                                 30   
30                                                 31   
31                                                 32   
32                                                 33   
33                                                 34   
34                                                 35   
35                                             Total#   
36                     *(Including foreign Nationals)   
37     **( more than 70% cases due to comorbidities )   
38  #States wise distribution is subject to furthe...   
39        #Our figures are being reconciled with ICMR   

                                   Name of State / UT  \
                                   Name of State / UT   
0                         Andaman and Nicobar Islands   
1                                      Andhra Pradesh   
2                                   Arunachal Pradesh   
3                                               Assam   
4                                               Bihar   
5                                          Chandigarh   
6                                        Chhattisgarh   
7            Dadra and Nagar Haveli and Daman and Diu   
8                                               Delhi   
9                                                 Goa   
10                                            Gujarat   
11                                            Haryana   
12                                   Himachal Pradesh   
13                                  Jammu and Kashmir   
14                                          Jharkhand   
15                                          Karnataka   
16                                             Kerala   
17                                             Ladakh   
18                                     Madhya Pradesh   
19                                        Maharashtra   
20                                            Manipur   
21                                          Meghalaya   
22                                            Mizoram   
23                                           Nagaland   
24                                             Odisha   
25                                         Puducherry   
26                                             Punjab   

In [5]:
table.columns = ["slno", "Region", "Active_Cases", "Active_New", "Cured/Discharged", "Cured_New", "Death", "Deaths_New"]
table.head()

,slno,Region,Active_Cases,Active_New,Cured/Discharged,Cured_New,Death,Deaths_New
0,1,Andaman and Nicobar Islands,822,9,648,148,20,NaN
1,2,Andhra Pradesh,87112,1626,138712,9097,2036,97
2,3,Arunachal Pradesh,670,14,1482,52,3,NaN
3,4,Assam,16367,616,42325,1734,145,5
4,5,Bihar,27906,1237,51158,2915,387,5


In [6]:
table["Date"] = [date.today().strftime("%d-%m-%Y") for i in range(len(table))]
table.drop(columns=["slno", "Active_New", "Cured_New", "Deaths_New"], inplace=True)
table.head()

,Region,Active_Cases,Cured/Discharged,Death,Date
0,Andaman and Nicobar Islands,822,648,20,10-08-2020
1,Andhra Pradesh,87112,138712,2036,10-08-2020
2,Arunachal Pradesh,670,1482,3,10-08-2020
3,Assam,16367,42325,145,10-08-2020
4,Bihar,27906,51158,387,10-08-2020


In [7]:
table = table.iloc[:-4,:]
table.iloc[-1,0] = "India"
table.tail()

F:\Anaconda\envs\webcrawler\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-7-a736726689be>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table.iloc[-1,0] = "India"


,Region,Active_Cases,Cured/Discharged,Death,Date
31,Tripura,1723,4443,42,10-08-2020
32,Uttarakhand,3373,6134,125,10-08-2020
33,Uttar Pradesh,47890,72650,2069,10-08-2020
34,West Bengal,26375,67120,2059,10-08-2020
35,India,634945,1535743,44386,10-08-2020


In [8]:
table["Confirmed_Cases"] = [table.iloc[i,1:-1].astype(int).sum() for i in range(len(table))]
table.head()

<ipython-input-8-20940eb6d4dd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table["Confirmed_Cases"] = [table.iloc[i,1:-1].astype(int).sum() for i in range(len(table))]


,Region,Active_Cases,Cured/Discharged,Death,Date,Confirmed_Cases
0,Andaman and Nicobar Islands,822,648,20,10-08-2020,1490
1,Andhra Pradesh,87112,138712,2036,10-08-2020,227860
2,Arunachal Pradesh,670,1482,3,10-08-2020,2155
3,Assam,16367,42325,145,10-08-2020,58837
4,Bihar,27906,51158,387,10-08-2020,79451


In [10]:
table.tail()

,Region,Active_Cases,Cured/Discharged,Death,Date,Confirmed_Cases
31,Tripura,1723,4443,42,10-08-2020,6208
32,Uttarakhand,3373,6134,125,10-08-2020,9632
33,Uttar Pradesh,47890,72650,2069,10-08-2020,122609
34,West Bengal,26375,67120,2059,10-08-2020,95554
35,India,634945,1535743,44386,10-08-2020,2215074


In [122]:
df = pd.read_csv("COVID_Database.csv")
df.head()

,Date,Region,Confirmed_Cases,Active_Cases,Cured/Discharged,Death
0,12-03-2020,India,74,71,3.0,0.0
1,13-03-2020,India,75,71,3.0,1.0
2,14-03-2020,India,84,72,10.0,2.0
3,15-03-2020,India,107,95,10.0,2.0
4,16-03-2020,India,114,99,13.0,2.0


In [23]:
%%writefile Update_Database_v3.py
from main_app_v2 import forecast_cases
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from bs4 import BeautifulSoup as bs
from datetime import date
from selenium import webdriver

def update(region_dict, driver):
    
    df = pd.read_html(driver.page_source)[0]
    
    df.columns = ["slno", "Region", "Active_Cases", "Active_New", "Cured/Discharged", "Cured_New", "Death", "Deaths_New"]
    df["Date"] = [date.today().strftime("%d-%m-%Y") for i in range(len(df))]
    df.drop(columns=["slno", "Active_New", "Cured_New", "Deaths_New"], inplace=True)
    df = df.iloc[:-4,:]
    df.iloc[-1,0] = "India"
    df["Confirmed_Cases"] = [df.iloc[i,1:-1].astype(int).sum() for i in range(len(df))]
    cols = list(df.columns)
    cols = [i.replace(" ","_") for i in cols]
    df.columns = cols
    #df["Date"] = [date.today().strftime('%Y-%m-%d') for i in range(len(df))]
    #df.index = pd.to_datetime(df.Date,format='%Y-%m-%d')
    df["Date"] = [date.today().strftime('%d-%m-%Y') for i in range(len(df))]
    df.index = pd.to_datetime(df.Date,format='%d-%m-%Y')
    for i,region in enumerate(region_dict):
        region_dict[region] = region_dict[region].append(df[df.Region == region], sort=False, ignore_index=True)
        try:
            region_dict[region].index = pd.to_datetime(region_dict[region].Date,format='%d-%m-%Y')
        except:
            region_dict[region].index = pd.to_datetime(region_dict[region].Date,format='%Y-%m-%d')
        
    return region_dict

def update_database():
    
    df = pd.read_csv("COVID_Database.csv")
    cols = list(df.columns)
    cols = [i.replace(" ","_") for i in cols]
    df.columns = cols

    states_df = df.groupby('Region')
    state_df = [pd.DataFrame(states_df.get_group(state)) for state in states_df.groups]
    
    #selenium setup
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome(r"C:\Users\Tejas\Downloads\chromedriver_win32\chromedriver.exe") #,chrome_options=options)
    driver.get("https://www.mohfw.gov.in/")
    
    region_dict = {}
    for region in states_df.groups:
        region_dict[region] = pd.DataFrame(states_df.get_group(region))
        try:
            region_dict[region].index = pd.to_datetime(region_dict[region].Date,format='%Y-%m-%d')
        except:
            region_dict[region].index = pd.to_datetime(region_dict[region].Date,format='%d-%m-%Y')

        region_dict = update(region_dict, driver)
    #     break
    
    driver.quit()
    
    df = pd.DataFrame()
    for region in region_dict.keys():
        df = df.append(region_dict[region], sort=False, ignore_index=True)

    cols = list(df.columns)
    cols = [i.replace(" ","_") for i in cols]
    df.columns = cols
    df.drop_duplicates(inplace=True)
    
    df.to_csv("COVID_Database.csv", index=False)
    
    return "Database Updated"


def forecasted_database():    
    df = pd.read_csv("COVID_Database.csv")
    df["Forecasted"] = ["" for i in range(len(df))]
    regions = sorted(list(df.Region.value_counts().keys()))

    df_old = pd.read_csv("Forecast.csv")
    
    forecast_df = pd.DataFrame(columns=['Date', 'Region', 'Confirmed_Cases', 'Active_Cases', 'Cured/Discharged',
           'Death', 'Forecasted', 'Forecasted_high', 'Forecasted_low', 'Confirmed_Cases_Daywise'])
    
    forecasts = []
    forecasts_high = []
    forecasts_low = []
    forecasts_daywise = []
    forecasts_daywise_high = []
    forecasts_daywise_low = []
    days = []
    region_x = []
#     forecasted = []
#     forecasted_region = []
    n=31
    for region in regions:
        try:
            forecast,std_error,confidence_interval = forecast_cases(region, n=n, return_all=True)

            forecast_old = df_old.Forecasted_Cases[df_old.Region==region].iloc[0]
            forecast_old = forecast[0]-forecast_old
            forecasts_daywise.append(forecast_old)
            forecasts_daywise_high.append(forecast_old+(forecast_old*0.05))
            forecasts_daywise_low.append(forecast_old-(forecast_old*0.05))

            forecast_high = confidence_interval[:,1]
            forecast_low = confidence_interval[:,0]
            forecasts.extend(forecast)
            forecasts_high.extend(forecast_high)
            forecasts_low.extend(forecast_low)
            forecasts_daywise.extend(pd.Series(forecast).diff(periods=1).iloc[1:])
            forecasts_daywise_high.extend(pd.Series(forecast_high).diff(periods=1).iloc[1:])
            forecasts_daywise_low.extend(pd.Series(forecast_low).diff(periods=1).iloc[1:])
            days.extend(pd.date_range(date.today(), periods=n).strftime('%d-%m-%Y'))
            region_x.extend([region for i in range(n)])
    #         forecasted.append(forecast[0])
    #         forecasted_region.append(region)
        except:
            pass

    forecast_df.Forecasted = forecasts
    forecast_df.Date = days
    forecast_df.Region = region_x
    forecast_df.Forecasted_high = forecasts_high
    forecast_df.Forecasted_low = forecasts_low
    forecast_df["Forecasted_Cases_Daywise"] = forecasts_daywise
    forecast_df["Forecasted_Cases_high_Daywise"] = forecasts_daywise_high
    forecast_df["Forecasted_Cases_low_Daywise"] = forecasts_daywise_low
    forecast_df.index = pd.to_datetime(forecast_df.Date)

    df2 = pd.concat([df,forecast_df], join='outer', ignore_index=True)
    df2 = df2.groupby("Region", as_index=False).apply(lambda x: x.iloc[:-1])
    df2.index = df2.Date 
    
    
    cols = ['Confirmed_Cases', 'Active_Cases', 'Cured/Discharged', 'Death']
    for col in cols:
        df2[col] = df2[col].shift(-1)
    df2 = df2.dropna(how="all",subset=['Confirmed_Cases', 'Active_Cases','Forecasted'])

    for region in regions:
        df2.Confirmed_Cases_Daywise[df2.Region==region] = df2.Confirmed_Cases[df2.Region==region].diff(periods=1)

    df2.to_csv("COVID_Forecasted.csv", index=False)

Writing Update_Database_v3.py


In [22]:
update_database()

'Database Updated'